In [0]:
%pip install pytest==8.4.2

In [0]:
from pyspark import pipelines as dp

In [0]:
# 🧰 Get configuration from pipeline parameters
catalog = spark.conf.get("catalog")
silver_schema = spark.conf.get("silver_schema")

print(f"Using catalog: {catalog}, silver_schema: {silver_schema}")

## Creating a Materialized View for Summarized Event Metrics

In this exercise, you will create a **materialized view** that summarizes key web event metrics from a dataset of website user interactions.  
This materialized view will help analysts monitor user behavior and funnel performance across different event types.

---

### Objective
Your task is to build a SQL query inside a **declarative pipeline materialized view** that performs the following:

1. Extracts and structures event-level data from the source table.
2. Calculates daily user engagement metrics.
3. Computes conversion funnel metrics.
4. Combines both sets of metrics into a single summarized result.

---

### Source Data

You will use the table:
{catalog}.{silver_schema}.web_site_events


This table contains the following relevant fields:

| Field Name      | Description |
|-----------------|--------------|
| `event_timestamp` | Timestamp of the event occurrence |
| `user_id`          | Unique identifier of the user |
| `session_id`       | Unique identifier of the user session |
| `device_type`      | Type of device used (e.g., desktop, mobile) |
| `product_id`       | ID of the product viewed or interacted with |
| `product_price`    | Price of the product at the time of the event |
| `event_type`       | Type of event (`view`, `click`, `add_to_cart`, `purchase`) |

---

### Prepare the Base Data

Create a **base CTE (Common Table Expression)** that:
- Converts the timestamp into a date column (`event_date`).
- Selects all necessary fields from the source table.

This step will make it easier to group and aggregate the data by day.

---

### Calculate User Session Metrics

From your base data, create a CTE that summarizes user engagement per day:

| Metric | Calculation | Description |
|---------|--------------|-------------|
| **`active_users`** | `COUNT(DISTINCT user_id)` | Number of unique users per day |
| **`total_sessions`** | `COUNT(DISTINCT session_id)` | Number of distinct sessions per day |
| **`avg_sessions_per_user`** | `total_sessions / active_users` | Average number of sessions per user per day |
| **`total_events`** | `COUNT(*)` | Total number of events on that day |
| **`avg_events_per_user`** | `total_events / active_users` | Average number of events per user per day |

Use **GROUP BY `event_date`** to aggregate the results by day.

---

### Calculate Funnel Metrics

Create another CTE that summarizes funnel performance for each day:

| Metric | Calculation | Description |
|---------|--------------|-------------|
| **`total_views`** | Count of events where `event_type = 'view'` | Number of product views |
| **`total_clicks`** | Count of events where `event_type = 'click'` | Number of product clicks |
| **`total_add_to_cart`** | Count of events where `event_type = 'add_to_cart'` | Number of items added to cart |
| **`total_purchases`** | Count of events where `event_type = 'purchase'` | Number of purchases completed |
| **`click_through_rate`** | `total_clicks / total_views` | Measures how often views lead to clicks |
| **`cart_abandonment_rate`** | `(total_add_to_cart - total_purchases) / total_add_to_cart` | Percentage of carts not converted into purchases |

*Tip:* Use `CASE WHEN` expressions to count specific event types and handle divisions safely (e.g., using `NULLIF` to avoid division by zero).

---

### Combine the Results

Join both CTEs (`user_sessions` and `funnel`) on the `event_date` column to create a single summarized dataset.  
Select all relevant metrics and order the results by `event_date` (descending).

---

### Expected Output Columns

| Column | Description |
|---------|--------------|
| `event_date` | Date of the events |
| `active_users` | Number of unique users |
| `total_sessions` | Number of sessions |
| `avg_sessions_per_user` | Average sessions per user |
| `total_events` | Total number of events |
| `avg_events_per_user` | Average events per user |
| `total_views` | Total number of view events |
| `total_clicks` | Total number of click events |
| `total_add_to_cart` | Total number of add-to-cart events |
| `total_purchases` | Total number of purchase events |
| `click_through_rate` | Ratio of clicks to views |
| `cart_abandonment_rate` | Ratio of abandoned carts to total carts |

---


In [0]:
@dp.materialized_view(
    name="mv_summarized_event_metrics"
)
def create_mv_summarized_event_metrics():
    return spark.sql(f"""
    WITH base AS (
        SELECT
            to_date(event_timestamp) AS event_date,
            user_id,
            session_id,
            device_type,
            product_id,
            product_price,
            event_type
        FROM {catalog}.{silver_schema}.web_site_events
    ),
    user_sessions AS (
        SELECT
            event_date,
            COUNT(DISTINCT user_id) AS active_users,
            COUNT(DISTINCT session_id) AS total_sessions,
            ROUND(COUNT(DISTINCT session_id)*1.0/COUNT(DISTINCT user_id),2) AS avg_sessions_per_user,
            COUNT(*) AS total_events,
            ROUND(COUNT(*)*1.0/COUNT(DISTINCT user_id),2) AS avg_events_per_user
        FROM base
        GROUP BY event_date
    ),
    funnel AS (
        SELECT
            event_date,
            SUM(CASE WHEN event_type='view' THEN 1 ELSE 0 END) AS total_views,
            SUM(CASE WHEN event_type='click' THEN 1 ELSE 0 END) AS total_clicks,
            SUM(CASE WHEN event_type='add_to_cart' THEN 1 ELSE 0 END) AS total_add_to_cart,
            SUM(CASE WHEN event_type='purchase' THEN 1 ELSE 0 END) AS total_purchases,
            ROUND(SUM(CASE WHEN event_type='click' THEN 1 ELSE 0 END)*1.0/NULLIF(SUM(CASE WHEN event_type='view' THEN 1 ELSE 0 END),1),2) AS click_through_rate,
            ROUND((SUM(CASE WHEN event_type='add_to_cart' THEN 1 ELSE 0 END) - SUM(CASE WHEN event_type='purchase' THEN 1 ELSE 0 END))*1.0/NULLIF(SUM(CASE WHEN event_type='add_to_cart' THEN 1 ELSE 0 END),1),2) AS cart_abandonment_rate
        FROM base
        GROUP BY event_date
    )
    SELECT
        u.event_date,
        u.active_users,
        u.total_sessions,
        u.avg_sessions_per_user,
        u.total_events,
        u.avg_events_per_user,
        f.total_views,
        f.total_clicks,
        f.total_add_to_cart,
        f.total_purchases,
        f.click_through_rate,
        f.cart_abandonment_rate
    FROM user_sessions u
    JOIN funnel f ON u.event_date = f.event_date
    ORDER BY u.event_date DESC
    """)

## Creating a Materialized View for Event Metrics by User

In this exercise, you will create a **materialized view** that summarizes user-level web event metrics from a dataset of website interactions.  
This materialized view will help analysts understand user behavior patterns, engagement intensity, and purchasing activity across different dates.

---

### Objective
Your task is to build a SQL query inside a **declarative pipeline materialized view** that performs the following:

1. Extracts and structures event-level data from the source table.
2. Calculates engagement and funnel metrics at the **user level**.
3. Adds a revenue aggregation based on product interactions.
4. Produces a summarized dataset that can be used for user analytics and segmentation.

---

### Source Data

You will use the table:
{catalog}.{silver_schema}.web_site_events

This table contains the following relevant fields:

| Field Name      | Description |
|-----------------|--------------|
| `event_timestamp` | Timestamp of the event occurrence |
| `user_id`          | Unique identifier of the user |
| `user_name`        | Name or identifier of the user (for reporting) |
| `session_id`       | Unique identifier of the user session |
| `device_type`      | Type of device used (e.g., desktop, mobile) |
| `product_id`       | ID of the product viewed or interacted with |
| `product_price`    | Price of the product at the time of the event |
| `event_type`       | Type of event (`view`, `click`, `add_to_cart`, `purchase`) |

---

### Prepare the Base Data

Create a **base CTE (Common Table Expression)** that:
- Converts the timestamp into a date column (`event_date`).
- Selects all necessary fields from the source table, including user identifiers and product details.

This will provide a clean, standardized structure for per-user aggregations.

---

### Calculate User-Level Metrics

From your base data, create a CTE that aggregates metrics **per user per day**.  
Each record in your output should represent the activity summary of one user on one date.

| Metric | Calculation | Description |
|---------|--------------|-------------|
| **`total_sessions`** | `COUNT(DISTINCT session_id)` | Number of unique sessions initiated by the user |
| **`total_events`** | `COUNT(*)` | Total number of events triggered by the user |
| **`total_views`** | Count of events where `event_type = 'view'` | Number of product views by the user |
| **`total_clicks`** | Count of events where `event_type = 'click'` | Number of product clicks by the user |
| **`total_add_to_cart`** | Count of events where `event_type = 'add_to_cart'` | Number of items added to the cart |
| **`total_purchases`** | Count of events where `event_type = 'purchase'` | Number of completed purchases |
| **`click_through_rate`** | `total_clicks / total_views` | Conversion ratio from views to clicks |
| **`cart_abandonment_rate`** | `(total_add_to_cart - total_purchases) / total_add_to_cart` | Ratio of carts not resulting in purchases |
| **`total_revenue`** | `SUM(product_price)` | Total product value associated with user actions |

*Tip:* Use conditional aggregation with `CASE WHEN` to count specific event types, and handle division safely using `NULLIF` to prevent division by zero.

---

### Combine and Order Results

Select all fields from your user-level CTE to form the final dataset.  
Order the results by `event_date` (descending), then by `user_id` and `user_name` for readability.

---

### Expected Output Columns

| Column | Description |
|---------|--------------|
| `user_id` | Unique identifier of the user |
| `user_name` | Name of the user |
| `event_date` | Date of the user activity |
| `total_sessions` | Number of sessions per user per day |
| `total_events` | Total number of events per user per day |
| `total_views` | Number of views per user per day |
| `total_clicks` | Number of clicks per user per day |
| `total_add_to_cart` | Number of add-to-cart events per user per day |
| `total_purchases` | Number of purchases per user per day |
| `click_through_rate` | Ratio of clicks to views |
| `cart_abandonment_rate` | Ratio of abandoned carts to total carts |
| `total_revenue` | Total revenue from the user’s actions per day |

---



In [0]:
@dp.materialized_view(
    name="mv_summarized_event_metrics_by_user"
)
def create_mv_event_metrics_by_user():
    return spark.sql(f"""
    WITH base AS (
        SELECT
            user_id,
            user_name,
            session_id,
            device_type,
            product_id,
            product_price,
            event_type,
            to_date(event_timestamp) AS event_date
        FROM {catalog}.{silver_schema}.web_site_events
    ),
    -- Sessions and events per user
    user_events AS (
        SELECT
            user_id,
            user_name,
            event_date,
            COUNT(DISTINCT session_id) AS total_sessions,
            COUNT(*) AS total_events,
            COUNT(CASE WHEN event_type='view' THEN 1 END) AS total_views,
            COUNT(CASE WHEN event_type='click' THEN 1 END) AS total_clicks,
            COUNT(CASE WHEN event_type='add_to_cart' THEN 1 END) AS total_add_to_cart,
            COUNT(CASE WHEN event_type='purchase' THEN 1 END) AS total_purchases,
            ROUND(COUNT(CASE WHEN event_type='click' THEN 1 END)*1.0/NULLIF(COUNT(CASE WHEN event_type='view' THEN 1 END),1),2) AS click_through_rate,
            ROUND((COUNT(CASE WHEN event_type='add_to_cart' THEN 1 END) - COUNT(CASE WHEN event_type='purchase' THEN 1 END))*1.0/NULLIF(COUNT(CASE WHEN event_type='add_to_cart' THEN 1 END),1),2) AS cart_abandonment_rate,
            SUM(product_price) AS total_revenue
        FROM base
        GROUP BY user_id, user_name, event_date
    )
    SELECT *
    FROM user_events
    ORDER BY event_date DESC, user_id, user_name;
    """)

## Creating a Materialized View for Event Metrics by Product

In this exercise, you will create a **materialized view** that summarizes product-level web event metrics from a dataset of website interactions.  
This materialized view will help analysts evaluate product engagement, conversion efficiency, and revenue performance over time.

---

### Objective
Your task is to build a SQL query inside a **declarative pipeline materialized view** that performs the following:

1. Extracts and structures event-level data for each product from the source table.  
2. Calculates daily engagement and funnel metrics by product.  
3. Adds a revenue aggregation to measure product-level performance.  
4. Produces a summarized dataset that can be used for trend analysis and product performance reporting.

---

### Source Data

You will use the table:
{catalog}.{silver_schema}.web_site_events

This table contains the following relevant fields:

| Field Name      | Description |
|-----------------|--------------|
| `event_timestamp` | Timestamp of the event occurrence |
| `product_id`       | Unique identifier of the product |
| `product_name`     | Name of the product |
| `product_category` | Product category or classification |
| `product_price`    | Price of the product at the time of the event |
| `user_id`          | Identifier of the user performing the event |
| `event_type`       | Type of event (`view`, `click`, `add_to_cart`, `purchase`) |

---

### Prepare the Base Data

Create a **base CTE (Common Table Expression)** that:
- Converts the timestamp into a date column (`event_date`).  
- Selects all necessary product and event fields.  
- Filters out any records with missing product information (`WHERE product_id IS NOT NULL`).

This ensures only valid product-level events are included in your analysis.

---

### Calculate Product-Level Metrics

From your base data, create a CTE that aggregates metrics **per product per day**.  
Each record should represent one product’s performance on a specific date.

| Metric | Calculation | Description |
|---------|--------------|-------------|
| **`total_events`** | `COUNT(*)` | Total number of events related to the product |
| **`unique_users`** | `COUNT(DISTINCT user_id)` | Number of distinct users interacting with the product |
| **`total_views`** | Count of events where `event_type = 'view'` | Total number of times the product was viewed |
| **`total_clicks`** | Count of events where `event_type = 'click'` | Total number of product clicks |
| **`total_add_to_cart`** | Count of events where `event_type = 'add_to_cart'` | Number of times the product was added to a cart |
| **`total_purchases`** | Count of events where `event_type = 'purchase'` | Number of completed purchases |
| **`click_through_rate`** | `total_clicks / total_views` | Conversion ratio from product views to clicks |
| **`cart_abandonment_rate`** | `(total_add_to_cart - total_purchases) / total_add_to_cart` | Percentage of carts not resulting in purchases |
| **`total_revenue`** | `SUM(product_price)` | Total revenue associated with the product on that date |

*Tip:*  
Use `CASE WHEN` logic for event-type counts and handle divisions carefully using `NULLIF` or `try_divide` to avoid division by zero errors.

---

### Combine and Order Results

Select all fields from your product-level CTE to form the final dataset.  
Order the results by `event_date` (descending) and `total_views` (descending) to highlight the most viewed products for each day.

---

### Expected Output Columns

| Column | Description |
|---------|--------------|
| `product_id` | Unique identifier of the product |
| `product_name` | Name of the product |
| `product_category` | Category of the product |
| `event_date` | Date of the product activity |
| `total_events` | Total number of events related to the product |
| `unique_users` | Number of unique users interacting with the product |
| `total_views` | Number of view events per product per day |
| `total_clicks` | Number of click events per product per day |
| `total_add_to_cart` | Number of add-to-cart events per product per day |
| `total_purchases` | Number of purchase events per product per day |
| `click_through_rate` | Ratio of clicks to views |
| `cart_abandonment_rate` | Ratio of abandoned carts to total carts |
| `total_revenue` | Total revenue from the product’s interactions per day |

---


In [0]:
@dp.materialized_view(
    name="mv_summarized_event_metrics_by_product"
)
def create_mv_event_metrics_by_product():
    return spark.sql(f"""
    WITH base AS (
        SELECT
            product_id,
            product_name,
            product_category,
            product_price,
            user_id,
            event_type,
            to_date(event_timestamp) AS event_date
        FROM {catalog}.{silver_schema}.web_site_events
        WHERE product_id IS NOT NULL
    ),
    product_events AS (
        SELECT
            product_id,
            product_name,
            product_category,
            event_date,
            COUNT(*) AS total_events,
            COUNT(DISTINCT user_id) AS unique_users,
            COUNT(CASE WHEN event_type='view' THEN 1 END) AS total_views,
            COUNT(CASE WHEN event_type='click' THEN 1 END) AS total_clicks,
            COUNT(CASE WHEN event_type='add_to_cart' THEN 1 END) AS total_add_to_cart,
            COUNT(CASE WHEN event_type='purchase' THEN 1 END) AS total_purchases,
            ROUND(try_divide(COUNT(CASE WHEN event_type='click' THEN 1 END)*1.0, COUNT(CASE WHEN event_type='view' THEN 1 END)),2) AS click_through_rate,
            ROUND((COUNT(CASE WHEN event_type='add_to_cart' THEN 1 END) - COUNT(CASE WHEN event_type='purchase' THEN 1 END))*1.0/NULLIF(COUNT(CASE WHEN event_type='add_to_cart' THEN 1 END),1),2) AS cart_abandonment_rate,
            SUM(product_price) AS total_revenue
        FROM base
        GROUP BY product_id, product_name, product_category, event_date
    )
    SELECT *
    FROM product_events
    ORDER BY event_date DESC, total_views DESC
    """)

## Creating a Materialized View for Fact Sales

In this exercise, you will create a **materialized view** that aggregates sales transaction data into a structured fact table.  
This view will provide a foundation for sales analytics, enabling metrics such as total sales value, quantity sold, and average unit price per product and user.

---

### Objective
Your task is to build a SQL query inside a **declarative pipeline materialized view** that performs the following:

1. Extracts and filters active sales transactions from the source table.  
2. Aggregates sales data by product, user, and sale date.  
3. Calculates key performance metrics, such as total quantity sold and total sales amount.  
4. Produces a summarized dataset suitable for downstream reporting or joining with dimension tables.

---

### Source Data

You will use the table:
{catalog}.{silver_schema}.sales

This table contains the following relevant fields:

| Field Name      | Description |
|-----------------|--------------|
| `_created_at` | Timestamp of the sale (used as sale date) |
| `sale_id` | Unique identifier of the sales transaction |
| `product_id` | Unique identifier of the product sold |
| `user_id` | Unique identifier of the customer |
| `quantity` | Number of units sold in the transaction |
| `price` | Unit price of the product at the time of sale |
| `_is_active` | Boolean flag indicating whether the record is active or valid |

---

### Prepare and Filter the Data

In your query:
- Filter the dataset to include only **active** records (`WHERE _is_active IS TRUE`).  
- Select the key attributes (`_created_at`, `product_id`, `user_id`, `sale_id`, `quantity`, `price`) for aggregation.  
- Use `_created_at` as the **sale date** to represent when each sale occurred.

---

### Calculate Sales Metrics

Aggregate the data **by sale date, product, and user** to compute the following metrics:

| Metric | Calculation | Description |
|---------|--------------|-------------|
| **`number_of_sales`** | `COUNT(sale_id)` | Total number of sales transactions per product and user |
| **`total_quantity_sold`** | `SUM(quantity)` | Total number of product units sold |
| **`total_sales_amount`** | `SUM(quantity * price)` | Total revenue generated by sales |
| **`avg_unit_price`** | `AVG(price)` | Average unit price across all transactions for that combination |

Use `ROUND()` to limit the average unit price to two decimal places for readability.

---

### Grouping and Output

Group the results by:
- `sale_date` (from `_created_at`)  
- `product_id`  
- `user_id`  

This ensures each record in the output represents a **unique product-user combination per day**.

---

### Expected Output Columns

| Column | Description |
|---------|--------------|
| `sale_date` | Date when the sale occurred |
| `product_id` | Unique identifier of the product sold |
| `user_id` | Unique identifier of the customer |
| `number_of_sales` | Number of sales transactions for the product-user combination |
| `total_quantity_sold` | Total quantity of items sold |
| `total_sales_amount` | Total monetary value of sales |
| `avg_unit_price` | Average unit price of the product |

---


In [0]:
@dp.materialized_view(
    name="mv_fact_sales"
)
def create_mv_fact_sales():
    return spark.sql(f"""
    SELECT
        _created_at AS sale_date,
        product_id,
        user_id,
        COUNT(sale_id) AS number_of_sales,
        SUM(quantity) AS total_quantity_sold,
        SUM(quantity * price) AS total_sales_amount,
        ROUND(AVG(price), 2) AS avg_unit_price
    FROM {catalog}.{silver_schema}.sales
    WHERE _is_active IS TRUE
    GROUP BY
        _created_at,
        product_id,
        user_id
    """)

## Creating a Materialized View for Product Dimension

In this exercise, you will create a **materialized view** that represents the **Product Dimension** table in your data model.  
This dimension will store unique product information, serving as a reference for analytical queries and joins with fact tables such as sales or event metrics.

---

### Objective
Your task is to build a SQL query inside a **declarative pipeline materialized view** that performs the following:

1. Extracts and deduplicates product information from the source table.  
2. Selects key product attributes that describe each item.  
3. Produces a clean and unique product dimension dataset suitable for use in a star schema model.

---

### Source Data

You will use the table:
{catalog}.{silver_schema}.products

This table contains the following relevant fields:

| Field Name | Description |
|-------------|-------------|
| `product_id` | Unique identifier of the product |
| `name` | Name of the product |
| `category` | Category or classification of the product |
| `price` | Standard or listed price of the product |

---

### Prepare and Deduplicate the Data

In your query:
- Select only the relevant descriptive columns for the product dimension.  
- Use the `DISTINCT` keyword to ensure that each `product_id` appears only once in the output.  
- Exclude any columns not required for product-level analytics (e.g., timestamps or audit fields).  

This ensures that the resulting dataset contains **one record per unique product**.

---

### Expected Output Columns

| Column | Description |
|---------|--------------|
| `product_id` | Unique identifier of the product |
| `name` | Name of the product |
| `category` | Category or type of the product |
| `price` | Product price as stored in the source system |

---


In [0]:
@dp.materialized_view(
    name="mv_dim_product"
)
def create_dim_product():
    return spark.sql(f"""
    SELECT
        DISTINCT
        product_id,
        name,
        category,
        price
    FROM {catalog}.{silver_schema}.products
    """)

## Creating a Materialized View for User Dimension

In this exercise, you will create a **materialized view** that represents the **User Dimension** table in your data model.  
This dimension will store unique, active user information, serving as a reference for analytical queries and joins with fact tables such as sales or event metrics.

---

### Objective
Your task is to build a SQL query inside a **declarative pipeline materialized view** that performs the following:

1. Extracts and filters active users from the source table.  
2. Selects key user attributes that describe each individual.  
3. Produces a clean and unique user dimension dataset suitable for use in a star schema model.

---

### Source Data

You will use the table:
{catalog}.{silver_schema}.users

This table contains the following relevant fields:

| Field Name | Description |
|-------------|-------------|
| `user_id` | Unique identifier of the user |
| `name` | Name of the user |
| `email` | Email address of the user |
| `phone` | Phone number of the user |
| `is_active` | Boolean flag indicating whether the user is active |

---

### Prepare and Filter the Data

In your query:
- Filter the dataset to include only **active users** (`WHERE is_active IS TRUE`).  
- Select only the relevant descriptive columns for the user dimension.  
- Use the `DISTINCT` keyword to ensure that each `user_id` appears only once in the output.  

This ensures that the resulting dataset contains **one record per active user**.

---

### Expected Output Columns

| Column | Description |
|---------|--------------|
| `user_id` | Unique identifier of the user |
| `name` | Name of the user |
| `email` | Email address of the user |
| `phone` | Phone number of the user |

---

In [0]:
@dp.materialized_view(
    name="mv_dim_user"
)
def create_dim_user():
    return spark.sql(f"""
    SELECT
        DISTINCT
        user_id,
        name,
        email,
        phone
    FROM {catalog}.{silver_schema}.users
    WHERE is_active IS TRUE
    """)

# How to Run the Lakeflow Declarative Pipelines in Databricks

To execute this Lakeflow Declarative Pipeline notebook:

1. Go to **Jobs & Pipelines** → **ETL Pipeline**.
2. **Enable the Lakehouse Flow editor** option.
3. Give your pipeline a **name**.
4. Choose the desired **catalog** (dev or prod) and the **schema** for the Gold layer.
5. Click on **Add existing assets**. Select the project_bundle folder as the root, and add this notebook as the source code path. Click on **Add**
6. In the **Pipeline settings** Select **Serverless** for the cluster type.
8. Create the 3 required configurations (catalog and schema)
9. Click **Run pipeline** to finish setup.

The pipeline will now run, creating all materialized views in the gold layer.

# Tests

TO DO:

- Check if there are any failed tests and investigate their root cause


In [0]:
from helpers import test_runner
import os

notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
os.environ["NOTEBOOK_NAME"] = notebook_path.split("/")[-1]

test_runner.run()